In [1]:
from data import get_loader, test_dataset
from options import opt
import torch.backends.cudnn as cudnn
import logging
from tensorboardX import SummaryWriter
from utils import clip_gradient, adjust_lr
from models.BBSNet_model import BBSNetChannelSpatialAttention, BBSNetTransformerAttention
from torchvision.utils import make_grid
from datetime import datetime
import numpy as np
import os
import torch
import torch.nn.functional as F
import sys
sys.path.append('./models')

from BBSNet_train import *
from BBSNet_test import *
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0
load data...
Start train...


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/home/mahdi/Documents/bach_prj/repo/BBS-Net/data.py", line 121, in __getitem__
    image = self.rgb_loader(self.images[index])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mahdi/Documents/bach_prj/repo/BBS-Net/data.py", line 154, in rgb_loader
    with open(path, 'rb') as f:
         ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '../RGBD/RGBD_dataset/NJU2000/RGB/000725_left.jpg'


# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [ ]:
# test(method='BBSNetChannelSpatialAttention')

In [ ]:
# evaluate()

In [ ]:
draw_curve()